# Introduction

This notebook demonstrates basic usage of BioThings Explorer, an engine for autonomously querying a distributed knowledge graph. BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](EXPLAIN_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

PREDICT queries are designed to **predict plausible relationships between one entity and an entity class**.  For example, in this notebook, we explore the question:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*"What drugs might be used to treat hyperphenylalaninemia?"*

**To experiment with an executable version of this notebook, [load it in Google Colaboratory](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/PREDICT_demo.ipynb).**

## Step 0: Load BioThings Explorer modules

First, install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [1]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer
!pip install git+https://github.com/biothings/biothings_schema.py#egg=biothings_schema

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [1]:
# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## Step 1: Find representation of "hyperphenylalaninemia" in BTE

In this step, BioThings Explorer translates our query string "hyperphenylalaninemia"  into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [2]:
ht = Hint()
# find all potential representations of hyperphenylalaninemia
hyperphenylalaninemia_hint = ht.query("hyperphenylalaninemia")
# select the correct representation of hyperphenylalaninemia
hyperphenylalaninemia = hyperphenylalaninemia_hint['DiseaseOrPhenotypicFeature'][0]
hyperphenylalaninemia

{'mondo': 'MONDO:0016543',
 'umls': 'C0751435',
 'name': 'hyperphenylalaninemia',
 'display': 'mondo(MONDO:0016543) umls(C0751435) name(hyperphenylalaninemia) ',
 'type': 'DiseaseOrPhenotypicFeature',
 'primary': {'identifier': 'mondo',
  'cls': 'DiseaseOrPhenotypicFeature',
  'value': 'MONDO:0016543'}}

## Step 2: Find drugs that are associated with genes which invovled in hyperphenylalaninemia

In this section, we find all paths in the knowledge graph that connect hyperphenylalaninemia to any entity that is a chemical compound.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

The parameters for `FindConnection` are described below:


In [3]:
help(FindConnection.__init__)

Help on function __init__ in module biothings_explorer.user_query_dispatcher:

__init__(self, input_obj, output_obj, intermediate_nodes, registry=None)
    Find relationships in the Knowledge Graph between an Input Object and an Output Object.
    
    params
    ------
    input_obj (required): must be an object returned from Hint corresponding to a specific biomedical entity.
                          Examples: 
            Hint().query("Fanconi anemia")['DiseaseOrPhenotypicFeature'][0]
            Hint().query("acetaminophen")['ChemicalSubstance'][0]
    
    output_obj (required): must EITHER be an object returned from Hint corresponding to a specific biomedical
                           entity, OR be a string or list of strings corresponding to Biolink Entity classes.
                           Examples:
            Hint().query("acetaminophen")['ChemicalSubstance'][0]
            'Gene'
            ['Gene','ChemicalSubstance']
    
    intermediate_nodes (required): the semantic

Here, we formulate a `FindConnection` query with "hyperphenylalaninemia" as the `input_ojb`, "ChemicalSubstance" as the `output_obj` (which corresponds to a Biolink Entity type).  We further specify with the `intermediate_nodes` parameter that we are looking for paths joining hyperphenylalaninemia and chemical compounds with *one* intermediate node that is a Gene.  (The ability to search for longer reasoning paths that include additional intermediate nodes will be added shortly.)

In [4]:
fc = FindConnection(input_obj=hyperphenylalaninemia, output_obj='ChemicalSubstance', intermediate_nodes=['Gene'])

We next execute the `connect` method, which performs the **query path planning** and **query path execution** process.  In short, BioThings Explorer is deconstructing the query into individual API calls, executing those API calls, then assembling the results.

A verbose log of this process is displayed below:

In [5]:
# set verbose=True will display all steps which BTE takes to find the connection
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'hyperphenylalaninemia' and 'ChemicalSubstance'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene




========== QUERY #1 -- fetch all Gene entities linked to hyperphenylalaninemia ==========

==== Step #1: Query path planning ====

Because hyperphenylalaninemia is of type 'DiseaseOrPhenotypicFeature', BTE will query our meta-KG for APIs that can take 'DiseaseOrPhenotypicFeature' as input

BTE found 3 apis:

API 1. semmeddisease(1 API call)
API 2. mydisease.info(1 API call)
API 3. biolink_disease2gene(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 2.1: http://mydisease.info/v1/query (POST "q=C0751435,C0751436&scopes=mondo.xrefs.umls,disgenet.xrefs.umls&fields=disgenet.genes_related_to_disease&species=human&size=100")
API 1.1: http://pending.biothings.io

## Step 3: Display and Filter results
This section demonstrates post-query filtering done in Python. Later, more advanced filtering functions will be added to the **query path execution** module for interleaved filtering, thereby enabling longer query paths. More details to come...

First, all matching paths can be exported to a data frame. Let's examine a sample of those results.

In [6]:
df = fc.display_table_view()

# because UMLS is not currently well-integrated in our ID-to-object translation system, removing UMLS-only outputs here
patternDel = "^umls:C\d+"
filter = df.output_id.str.contains(patternDel)
df = df[~filter]

df.sample(10)

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output_id,output_name,output_type
638,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:5805,PTS,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1614854,DEXTROSE,ChemicalSubstance
936,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5053,PAH,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL1201775,SAPROPTERIN DIHYDROCHLORIDE,ChemicalSubstance
960,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5805,PTS,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL253582,SUCROSE,ChemicalSubstance
520,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5805,PTS,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL92,DOCETAXEL,ChemicalSubstance
617,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:3308,HSPA4,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL469912,PUROMYCIN,ChemicalSubstance
636,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,,entrez:5805,PTS,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1614854,DEXTROSE,ChemicalSubstance
1049,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5053,PAH,Gene,molecularlyInteractsWith,semmed,semmedgene,,drugbank:DB01592,None,ChemicalSubstance
696,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5053,PAH,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL1201774,SAPROPTERIN,ChemicalSubstance
7,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:773,CACNA1A,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL2103836,IMAGABALIN,ChemicalSubstance
786,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:5053,PAH,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL292439,NORLEUCINE,ChemicalSubstance


While most results are based on edges from [semmed](https://skr3.nlm.nih.gov/SemMed/), edges from [DGIdb](http://www.dgidb.org/), [biolink](https://monarchinitiative.org/), [disgenet](http://www.disgenet.org/), and [mychem.info](https://mychem.info) were also retrieved from their respective APIs. 

Next, let's examine which chemical compounds were most frequently mentioned.

In [7]:
df.output_name.value_counts().head(10)

DEXTROSE          13
SAPROPTERIN       12
NOREPINEPHRINE    12
DOPAMINE          10
PHENYLALANINE      8
NITROGEN           8
NITRIC OXIDE       8
DROXIDOPA          8
HYDROCORTISONE     8
GUANINE            6
Name: output_name, dtype: int64

Hyperphenylalaninemia is a condition characterized by elevated levels of phenylalanine in the blood. This phenotype is strongly associated with phenylketonuria (PKU), an inherited, genetic disorder that affects the ability to metabolize phenylalanine.  **Sapropterin** is a naturally-occurring cofactor associated with several enzymatic processes, including the metabolism of phenylalanine to tyrosine. It has been [FDA-approved](https://www.centerwatch.com/drug-information/fda-approved-drugs/drug/972/kuvan-sapropterin-dihydrochloride) as a treatment for PKU patients.  Tyrosine is also a precursor to several neurotransmitters, including **norepinephrine** and **dopamine**.

While several of the listed compounds would be considered positive controls, others on the list could be viewed as **testable hypotheses and discovery opportunities** to be evaluated by domain experts. For example, let's examine which genes link hyperphenylalaninemia to nitric oxide.

In [8]:
df[df.output_name=="NITRIC OXIDE"]

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output_id,output_name,output_type
242,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,,entrez:2643,GCH1,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
243,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,,entrez:2643,GCH1,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
244,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:2643,GCH1,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
245,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:2643,GCH1,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
246,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:2643,GCH1,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
247,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:2643,GCH1,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
554,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:3308,HSPA4,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance
555,hyperphenylalaninemia,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:3308,HSPA4,Gene,molecularlyInteractsWith,semmed,semmedgene,,chembl:CHEMBL1200689,NITRIC OXIDE,ChemicalSubstance


BTE identifies the genes [HSPA4](https://www.ncbi.nlm.nih.gov/gene/3308) and [GCH1](https://www.ncbi.nlm.nih.gov/gene/2643) as being the key nodes that join hyperphenylalaninemia with nitric oxide.

## Conclusions and caveats

This notebook demonstrated the use of BioThings Explorer in PREDICDT mode to identify chemical compounds related to hyperphenylalaninemia. BioThings Explorer autonomously queried a **distributed knowledge graph of biomedical APIs** to find the most common compounds, and several of the top hits had a clear biochemical and clinical relevance to hyperphenylalaninemia.

There are still many areas for improvement (and some areas in which BioThings Explorer is still buggy).  And of course, BioThings Explorer is dependent on the accessibility of the APIs that comprise the distributed knowledge graph.  Nevertheless, we encourage users to try other variants of the PREDICT queries demonstrated in this notebook.